In [15]:
!pip install --upgrade tables
!pip install eli5
!pip install xgboost

Requirement already up-to-date: tables in /usr/local/lib/python3.6/dist-packages (3.6.1)


In [0]:
import pandas  as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score, KFold

import eli5
from eli5.sklearn import PermutationImportance

In [17]:
cd '/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_two/dw_matrix_car'

/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_two/dw_matrix_car


In [0]:
df = pd.read_hdf('data/car.h5')

In [0]:
SUFFIX_CAT = '__cat'
for feat in df.columns:
  if isinstance(df[feat][0],list): continue
  
  factorized_values = df[feat].factorize()[0]
  if SUFFIX_CAT in feat:
    df[feat] = factorized_values
  else:
    df[feat+SUFFIX_CAT] = factorized_values


In [63]:
cat_feats = [x for x in df.columns if SUFFIX_CAT in x]
cat_feats = [x for x in cat_feats if 'price' not in x]
len(cat_feats)

151

In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 106494 entries, 0 to 160615
Columns: 309 entries, breadcrumb to car_id__cat
dtypes: bool(70), float64(1), int64(156), object(82)
memory usage: 207.1+ MB


In [0]:
def run_model(model,feats):
  X = df[feats].values
  y=df['price_value'].values


  model = model
  scores = cross_val_score(model,X,y,cv=3,scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

# Decision Tree

In [24]:
run_model(DecisionTreeRegressor(max_depth=5),cat_feats)

(-19695.13091100928, 148.72570644015792)

#Random Forest

In [25]:
model = RandomForestRegressor(max_depth=5,n_estimators=50,random_state=0)
run_model(model,cat_feats)

(-18718.657185256638, 64.5424578125788)

#XGBOOST

In [28]:
xgb_params = {
    'max_depth':5,
    'n_estimarots' : 50,
    'learning_rate' : 0.1,
    'seed' : 0
}
model = xgb.XGBRegressor(**xgb_params)
run_model(model,cat_feats)

[08:58:09] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:58:48] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:59:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-11862.434165529698, 264.2411021793361)

In [29]:
model = xgb.XGBRegressor(**xgb_params)
model.fit(X,y)

imp = PermutationImportance(model,random_state=0).fit(X,y)
eli5.show_weights(imp, feature_names=cat_feats)

[09:01:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Weight,Feature
0.1341 ± 0.0029,param_rok-produkcji__cat
0.1037 ± 0.0028,param_stan__cat
0.0956 ± 0.0029,param_napęd__cat
0.0501 ± 0.0020,param_skrzynia-biegów__cat
0.0442 ± 0.0014,param_moc__cat
0.0438 ± 0.0018,param_faktura-vat__cat
0.0401 ± 0.0008,param_marka-pojazdu__cat
0.0216 ± 0.0007,param_typ__cat
0.0182 ± 0.0012,feature_kamera-cofania__cat
0.0175 ± 0.0006,param_wersja__cat


In [46]:
feats = ['param_rok-produkcji__cat',
'param_stan__cat',
'param_napęd__cat',
'param_skrzynia-biegów__cat',
'param_moc__cat',
'param_faktura-vat__cat',
'param_marka-pojazdu__cat',
'param_typ__cat',
'feature_kamera-cofania__cat',
'param_wersja__cat',
'param_model-pojazdu__cat',
'param_pojemność-skokowa__cat',
'param_kod-silnika__cat',
'seller_name__cat',
'feature_wspomaganie-kierownicy__cat',
'feature_czujniki-parkowania-przednie__cat',
'param_uszkodzony__cat',
'feature_system-start-stop__cat',
'feature_regulowane-zawieszenie__cat',
'feature_asystent-pasa-ruchu__cat']

run_model(model,feats)

[11:07:44] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[11:07:53] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[11:08:02] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-11861.928807244389, 100.50210848351769)

In [0]:
df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x: -1 if str(x)== 'None' else int(x))


In [47]:
feats = ['param_rok-produkcji',
'param_stan__cat',
'param_napęd__cat',
'param_skrzynia-biegów__cat',
'param_moc__cat',
'param_faktura-vat__cat',
'param_marka-pojazdu__cat',
'param_typ__cat',
'feature_kamera-cofania__cat',
'param_wersja__cat',
'param_model-pojazdu__cat',
'param_pojemność-skokowa__cat',
'param_kod-silnika__cat',
'seller_name__cat',
'feature_wspomaganie-kierownicy__cat',
'feature_czujniki-parkowania-przednie__cat',
'param_uszkodzony__cat',
'feature_system-start-stop__cat',
'feature_regulowane-zawieszenie__cat',
'feature_asystent-pasa-ruchu__cat']
run_model(model,feats)

[11:08:58] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[11:09:07] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[11:09:16] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-10067.086619567794, 35.57616233370741)

In [0]:
df['param_moc']= df['param_moc'].map(lambda x:-1 if str(x)== 'None' else int(x.split(' ')[0]) )

In [56]:
feats = ['param_rok-produkcji',
'param_stan__cat',
'param_napęd__cat',
'param_skrzynia-biegów__cat',
'param_moc',
'param_faktura-vat__cat',
'param_marka-pojazdu__cat',
'param_typ__cat',
'feature_kamera-cofania__cat',
'param_wersja__cat',
'param_model-pojazdu__cat',
'param_pojemność-skokowa__cat',
'param_kod-silnika__cat',
'seller_name__cat',
'feature_wspomaganie-kierownicy__cat',
'feature_czujniki-parkowania-przednie__cat',
'param_uszkodzony__cat',
'feature_system-start-stop__cat',
'feature_regulowane-zawieszenie__cat',
'feature_asystent-pasa-ruchu__cat']
run_model(model,feats)

[11:16:28] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[11:16:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[11:16:44] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-8698.488115726954, 52.63025457895438)

In [0]:
df['param_pojemność-skokowa'] = df['param_pojemność-skokowa'].map(lambda x:-1 if str(x)== 'None' else int(x.split('cm')[0].replace(' ','')) )

In [68]:
feats = ['param_rok-produkcji',
'param_stan__cat',
'param_napęd__cat',
'param_skrzynia-biegów__cat',
'param_moc',
'param_faktura-vat__cat',
'param_marka-pojazdu__cat',
'param_typ__cat',
'feature_kamera-cofania__cat',
'param_wersja__cat',
'param_model-pojazdu__cat',
'param_pojemność-skokowa',
'param_kod-silnika__cat',
'seller_name__cat',
'feature_wspomaganie-kierownicy__cat',
'feature_czujniki-parkowania-przednie__cat',
'param_uszkodzony__cat',
'feature_system-start-stop__cat',
'feature_regulowane-zawieszenie__cat',
'feature_asystent-pasa-ruchu__cat']
run_model(model,feats)

[11:29:09] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[11:29:18] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[11:29:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-8580.61650667904, 58.180956552556225)